In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import MSELoss
import pandas as pd
from AttentiveFP import get_smiles_dicts, get_smiles_array, num_atom_features, num_bond_features  # assumed to be your actual featurizer
from AttentiveFP import Fingerprint  # assumed to be your attentive FP model

In [2]:
# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'Endocrine_Disruption_NR-AhR'

# --- Dataset ---
class MoleculeDataset(Dataset):
    def __init__(self, smiles_list, targets, feature_dicts):
        self.smiles_list = smiles_list
        self.targets = targets
        self.feature_dicts = feature_dicts
        self.x_atom, self.x_bond, self.x_atom_index, self.x_bond_index, self.x_mask, _ = get_smiles_array(smiles_list, feature_dicts)

    def __len__(self):
        return len(self.feature_dicts)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.x_atom[idx], dtype=torch.float32),
            torch.tensor(self.x_bond[idx], dtype=torch.float32),
            torch.tensor(self.x_atom_index[idx], dtype=torch.long),
            torch.tensor(self.x_bond_index[idx], dtype=torch.long),
            torch.tensor(self.x_mask[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

In [3]:
# --- Load data ---
df = pd.read_csv(f"../data/{model_name}.csv")
smiles_list = df["smiles"].tolist()
targets = df["active"].tolist()
feature_dicts = get_smiles_dicts(smiles_list)

# --- Dataloader ---
dataset = MoleculeDataset(smiles_list, targets, feature_dicts)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
# --- Model ---
model = Fingerprint(
    radius=5,
    T=3,
    input_feature_dim=num_atom_features(),
    input_bond_dim=num_bond_features(),
    fingerprint_dim=200,
    output_units_num=1,
    p_dropout=0.1
).to(device)

optimizer = Adam(model.parameters(), lr=1e-3)
loss_fn = MSELoss()

In [5]:
from sklearn.model_selection import train_test_split

# Split SMILES and targets
train_smiles, val_smiles, train_targets, val_targets = train_test_split(
    smiles_list, targets, test_size=0.2, random_state=42
)

train_set = MoleculeDataset(train_smiles, train_targets, feature_dicts)
val_set = MoleculeDataset(val_smiles, val_targets, feature_dicts)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)


In [6]:
import os

# Folder to save model
os.makedirs("../models", exist_ok=True)
best_loss = float("inf")

try:
    # Load existing model
    model.load_state_dict(torch.load(f"../models/model_{model_name}.pt"))
    model.to(device)
    model.eval()
except:
    print("The model could not be loaded. Training a new model from scratch.")

best_loss = float("inf")
os.makedirs("../models", exist_ok=True)

for epoch in range(30):
    model.train()
    train_loss = 0

    for atom, bond, atom_deg, bond_deg, mask, target in train_loader:
        atom, bond, atom_deg, bond_deg, mask, target = [
            t.to(device) for t in (atom, bond, atom_deg, bond_deg, mask, target)
        ]
        _, pred, _ = model(atom, bond, atom_deg, bond_deg, mask)
        loss = loss_fn(pred.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for atom, bond, atom_deg, bond_deg, mask, target in val_loader:
            atom, bond, atom_deg, bond_deg, mask, target = [
                t.to(device) for t in (atom, bond, atom_deg, bond_deg, mask, target)
            ]
            _, pred, _ = model(atom, bond, atom_deg, bond_deg, mask)
            loss = loss_fn(pred.squeeze(), target)
            val_loss += loss.item()

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Save best model
    if val_loss < best_loss:
        best_loss = val_loss
        path = f"../models/model_{model_name}.pt"
        torch.save(model.state_dict(), path)
        print(f"✅ Saved best model to {path} (val loss: {best_loss:.4f})")



RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.